# LangGraph로 기본 챗봇 만들기

이 노트북에서는 **LangGraph**를 사용하여 **기본적인 챗봇 아키텍처**를 구현합니다.

## ch04 복습: LangGraph의 구성요소

```
┌────────────────────────────────────────────────────────────────────┐
│                 LangGraph의 3가지 핵심 요소                         │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   1️⃣ State (상태)                                                 │
│      • 그래프 전체에서 공유되는 데이터                              │
│      • 대화 기록, 중간 결과 등 저장                                │
│                                                                    │
│   2️⃣ Node (노드)                                                  │
│      • 실제 작업을 수행하는 함수                                   │
│      • LLM 호출, 데이터 처리 등                                    │
│                                                                    │
│   3️⃣ Edge (엣지)                                                  │
│      • 노드 간의 연결                                              │
│      • 실행 순서 결정                                              │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 이 노트북에서 만들 챗봇 구조

```
┌────────────────────────────────────────────────────────────────────┐
│                    기본 챗봇 아키텍처                               │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│          ┌─────────┐      ┌─────────┐      ┌─────────┐            │
│          │  START  │ ───▶ │ chatbot │ ───▶ │   END   │            │
│          └─────────┘      └─────────┘      └─────────┘            │
│                                │                                   │
│                                ▼                                   │
│                        ┌─────────────┐                             │
│                        │    LLM      │                             │
│                        │   호출      │                             │
│                        └─────────────┘                             │
│                                                                    │
│   가장 단순한 형태의 챗봇:                                          │
│   사용자 입력 → LLM 처리 → 응답 반환                               │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama langgraph

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. State (상태) 정의

챗봇이 관리할 데이터 구조를 정의합니다.

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph import add_messages

class State(TypedDict):
    """
    챗봇의 상태를 정의합니다.
    
    messages: 대화 기록 리스트
    - Annotated[list, add_messages]: 
      새 메시지가 오면 덮어쓰지 않고 추가함
    """
    messages: Annotated[list, add_messages]

print("✅ State 정의 완료")
print("   messages: 대화 기록을 저장하는 리스트")

# 3. Node (노드) 정의

LLM을 호출하는 챗봇 노드를 만듭니다.

In [ ]:
from langchain_ollama import ChatOllama

# LLM 모델
model = ChatOllama(model='llama3.2')

def chatbot(state: State):
    """
    챗봇 노드
    
    1. 현재 상태에서 메시지 기록을 가져옴
    2. LLM에게 전달하여 응답 생성
    3. 응답을 메시지 리스트에 추가하여 반환
    """
    # LLM 호출
    answer = model.invoke(state['messages'])
    
    # 응답을 메시지에 추가
    return {'messages': [answer]}

print("✅ chatbot 노드 정의 완료")

# 4. Graph (그래프) 구성

노드와 엣지를 연결하여 그래프를 완성합니다.

In [ ]:
from langgraph.graph import StateGraph, START, END

# 그래프 빌더 생성
builder = StateGraph(State)

# 노드 추가
# 첫 번째 인자: 노드 이름 (고유해야 함)
# 두 번째 인자: 실행할 함수
builder.add_node('chatbot', chatbot)

# 엣지 추가
builder.add_edge(START, 'chatbot')  # 시작 → 챗봇
builder.add_edge('chatbot', END)     # 챗봇 → 종료

# 그래프 컴파일
graph = builder.compile()

print("✅ 그래프 컴파일 완료")

# 5. 그래프 구조 시각화

In [ ]:
# 그래프 구조를 Mermaid 형식으로 출력
print("=== 그래프 구조 (Mermaid) ===")
print(graph.get_graph().draw_mermaid())

# 6. 그래프 실행

## stream vs invoke

| 메서드 | 동작 | 사용 시나리오 |
|--------|------|---------------|
| **invoke** | 최종 결과만 반환 | 결과만 필요할 때 |
| **stream** | 단계별 결과 반환 | 진행 상황을 보고 싶을 때 |

In [ ]:
from langchain_core.messages import HumanMessage

# 입력 준비
input_data = {'messages': [HumanMessage(content='안녕하세요!')]}

print("=== stream으로 실행 ===")
print(f"입력: {input_data['messages'][0].content}\n")

# stream으로 실행 (단계별 출력)
for chunk in graph.stream(input_data):
    print(f"chunk: {chunk}")

In [ ]:
# invoke로 실행 (최종 결과만)
result = graph.invoke(input_data)

print("\n=== invoke로 실행 ===")
print(f"응답: {result['messages'][-1].content}")

# 7. 다양한 질문으로 테스트

In [ ]:
test_questions = [
    "파이썬이란 무엇인가요?",
    "오늘 날씨가 어때요?",
    "간단한 인사말을 알려주세요.",
]

for question in test_questions:
    print(f"\n{'='*50}")
    print(f"사용자: {question}")
    
    result = graph.invoke({'messages': [HumanMessage(content=question)]})
    
    print(f"\nAI: {result['messages'][-1].content[:200]}...")

---

## 정리: 기본 챗봇 아키텍처

### 구현 단계

```python
# 1. State 정의
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 2. Node 정의
def chatbot(state: State):
    answer = model.invoke(state['messages'])
    return {'messages': [answer]}

# 3. Graph 구성
builder = StateGraph(State)
builder.add_node('chatbot', chatbot)
builder.add_edge(START, 'chatbot')
builder.add_edge('chatbot', END)

# 4. 컴파일 및 실행
graph = builder.compile()
result = graph.invoke({'messages': [...]})
```

### 핵심 개념

| 개념 | 역할 | 비유 |
|------|------|------|
| **State** | 공유 데이터 | 공유 메모장 |
| **Node** | 작업 수행 | 작업자 |
| **Edge** | 노드 연결 | 작업 순서표 |
| **START/END** | 시작/종료점 | 출발/도착 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model='gpt-4o-mini')

# 변경
from langchain_ollama import ChatOllama
model = ChatOllama(model='llama3.2')
```

## 다음 단계

**여러 노드를 연결**하여 SQL 쿼리를 생성하고 설명하는 시스템을 만듭니다. (04-05번 노트북)